# Classificador KNN - MNIST

Estamos interessados em reconhecer digitos utilizando nosso classificador KNN, promovendo uma seleção automática de atributos com o intuito de diminuir a dimensionalidade do problema e melhorar o desempenho do nosso classificador.

## Importando bibliotecas

In [ ]:
# Para obter o data set de imagens
from tensorflow.examples.tutorials.mnist import input_data
# Biblioteca numpy
import numpy as np
# Classificador KNN
from classifiers.neighbors import KNeighborsClassifier
# Para medir o tempo de execução dos algoritmos
from ext.timer import elapsed_timer
# Para realizar a seleção dos atributos
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.ensemble import ExtraTreesClassifier
# Para dividir o data set 
from sklearn.model_selection import train_test_split

## Obtendo o data set

A seguir, utilizamos a biblioteca _tensorflow_ para obter o data set MNIST

In [ ]:
mnist = input_data.read_data_sets('datasets/MNIST_data/')

In [ ]:
# Define o tamanho do data set:
TAMANHO = 6000

In [ ]:
def get_data():
    X = np.asarray(mnist.train.images[:TAMANHO])
    y = np.asarray(mnist.train.labels[:TAMANHO])
    return X, y

## Classificação

### Definindo uma função que exibe os resultados

In [ ]:
def mostrar_resultados(test_images, test_labels, pred):
    i = 0
    total_correct = 0
    for test_image in test_images:
        if pred[i] == test_labels[i]:
            total_correct += 1
        acc = (total_correct / (i+1)) * 100
        print('test image['+str(i)+']', '\tpred:', pred[i], '\torig:', test_labels[i], '\tacc:', str(round(acc, 2))+'%')
        i += 1

### Definindo uma função de classificação

In [ ]:
def classificar(train_images, train_labels, test_images, test_labels):
    with elapsed_timer() as elapsed:
        classifier = KNeighborsClassifier(leaf_size=30, n_neighbors=3, algorithm='kd_tree')
        classifier.fit(train_images, train_labels)
        pred = classifier.predict(test_images)
        print("Tempo de execução: " + str(elapsed()))
    mostrar_resultados(test_images, test_labels, pred)

### Classificando imagens 

A seguir instanciamos o nosso classificador knn com os seguintes hiper-parâmetros:
- Leaf size = 30
- n_neighbors = 3
- dist_metric = euclidean
- weights = distance

Iremos promover e testar a seleção de atributos utilizando os seguintes métodos:
- Univariate Selection
- Feature Importance

In [ ]:
# Classificação com todos os atributos:
X, y = get_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

classificar(X_train, y_train, X_test, y_test)

### Univariate Selection

Esse algoritmo procura selecionar os algoritmos que promovem relações mais fortes com a variável de saída. O parâmetro ```k``` em ```SelectKBest``` especifica quantos atributos queremos manter no data set.

Note que cada instância contém 768 atributos, e portanto devemos procurar um número adequado de atributos que não prejudiquem a representatividade da imagem.

Talvez possamos calcular a média dos scores dos atributos após a aplicação do método, e então selecionar apenas os atributos que tenham um score maior que a média.

In [ ]:
X, y = get_data()

# Extração de atributos
test = SelectKBest(score_func=chi2, k=768)
fit = test.fit(X, y)

np.set_printoptions(precision=3)
X = fit.transform(X)

# Calcula a média dos scores
fit_scores = np.asarray(fit.scores_)
mean = np.nanmean(fit_scores)
# Quantidade de atributos:
print((fit_scores > mean).sum())

In [ ]:
# Executa novamente:

X, y = get_data()

test = SelectKBest(score_func=chi2, k=289)
fit = test.fit(X, y)
np.set_printoptions(precision=3)
X = fit.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Testa o desempenho do classificador:
classificar(X_train, y_train, X_test, y_test)

Houve pouca diferença quanto ao desempenho do método, mas considere que diminuimos a dimensão por mais da metade. A velocidade com uma quantidade de menor dimensões é reduzida. 

É razoável supor que com mais alguns ajustes o desempenho do algoritmo melhore, pois supomos uma heurística para resolver o problema e não testamos outra quantidade de seleção de atributos.

### Feature Importance

In [ ]:
X, y = get_data()

model = ExtraTreesClassifier()
model.fit(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Testa o desempenho do classificador:
classificar(X_train, y_train, X_test, y_test)

## Referencias

Rahul Bhalley. __Digit recognition__. 2017. _https://towardsdatascience.com/mnist-with-k-nearest-neighbors-8f6e7003fab7_. 

Jason Brownlee. __Feature Selection For Machine Learning in Python__. May 20, 2016. Disponível em _https://machinelearningmastery.com/feature-selection-machine-learning-python/_.